# Repeated Classes in PMO 

In [5]:
import datetime
import pathlib
import urllib.parse
import xml.etree.ElementTree as etree

import rdflib

from jinja2 import Template

DC = rdflib.Namespace("http://purl.org/dc/terms/")
OWL = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
RDACT = rdflib.Namespace("http://rdaregistry.info/termList/RDACarrierType/")
SKOS = rdflib.Namespace("http://www.w3.org/2004/02/skos/core#")

version_2_0 = pathlib.Path("/Users/jpnelson/30-39 LD4P, PCC, Sinopia, and FOLIO/36.01 LD4P Performed Music Ontology/2.0")

In [6]:
pmo_rdf_xml = version_2_0 / "Ontology/PMO.rdf"

In [7]:
pmo_rdf_xml.exists()

True

In [8]:
pmo_graph = rdflib.Graph()
pmo_graph.parse(version_2_0 / "Ontology/PMO.rdf")

<Graph identifier=Na1885007952349d28d9d303c7061f889 (<class 'rdflib.graph.Graph'>)>

In [22]:
pmo_classes = [subj for subj in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.Class)]

In [13]:
template = Template("""<?xml version="1.0" encoding="UTF-8"?>
<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <link href="/css/owl.css" rel="stylesheet" type="text/css" />
    <link href="/css/Primer.css" rel="stylesheet" type="text/css" />
    <link href="/css/rec.css" rel="stylesheet" type="text/css" />
    <link href="/css/extra.css" rel="stylesheet" type="text/css" />
    <link rel="shortcut icon" href="/css/favicon.ico" />
    <script src="/js/jquery.js"></script>
    <script src="/js/jquery.scrollTo.js"></script>
    <script src="/PerformedMusicOntology/js/marked.min.js"></script>
    <script>
      $(document).ready(
            function () {
                jQuery(".markdown").each(function(el){
                    jQuery(this).after(marked(jQuery(this).text())).remove()});
                var list = $('a[name="http://www.essepuntato.it/static/tmp/0"]');
                if (list.size() != 0) {
                    var element = list.first();
                    $.scrollTo(element);
                }
        });
    </script>
  </head>
  <body>
    <div class="head">
      <h1>Performed Music Ontology 2.0</h1>
      <dl>
        <dt>IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
        <dt>Version IRI:</dt>
        <dd>http://performedmusicontology.org/2.0/ontology/</dd>
       </dl>
       <dl>
         <dt>Other visualisation:</dt>
         <dd><a href="http://performedmusicontology.org/2.0/ontology/PerformedMusicOntology.rdf">Ontology source</a></dd>
        </dl>
     </div>
     <hr />
     <div id="toc">
       <h2>Table of Content</h2>
       <ol>
          <li><a href="#classes">Classes</a></li>
          <li><a href="#objectproperties">Object Properties</a></li>
          <li><a href="#dataproperties">Data Properties</a></li>
          <li><a href="#namespacedeclarations">Namespace Declarations</a></li>
        </ol>
     </div>
     <div id="classes">
        <h2>Classes</h2>
        <ul class="hlist">
          {% for class_ in classes %}
            <li>
              <a href="#{{ class_.id }}" title="{{ class_.uri }}">
                <span>{{ class_.label }}</span>
              </a>
            </li>
          {% endfor %}
        </ul>
        {% for class_ in classes %}
        <div id="{{ class_.id }}" class="entity">
          <a name="{{ class_.uri }}"></a>
          <h3>
           {{ class_.label }}<sup title="class" class="type-c">c</sup>
           <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#classes">Class ToC</a></span>
          </h3>
          <p><strong>IRI: </strong>{{ class_.uri }}</p>
          <div class="comment">
           {% if class_.definition %}
            <span class="markdown">{{ class_.definition }}</span>
           {% endif %}
           {% if class_.example %}
            {% for example in class_.example %}
            <span class="markdown">{{ example }}</span>
            {% endfor %}
           {% endif %}
          </div>
          {% if class_.superClass or class_.children %}
           <dl class="description">
           {% if class_.superClass %}
             <dt>has super-classes</dt>
             {% for row in class_.superClass %}
             <dd>
               <span class="dotted" title="{{ row.uri }}">{{ row.label }}</span>
             </dd>
             {% endfor %}
           {% endif %}
           {% if class_.children %}
            <dt>has sub-classes</dt>
            {% for child in class_.children %}
            <dd>
               <a href="{{ child.uri }}">{{ child.label }}</a>
               <sup title="class" class="type-c">c</sup>{% if not loop.last %},{% endif %}
            </dd>
            {% endfor %}
           {% endif %}
           </dl>
          {% endif %}
        </div>
        {% endfor %}
        <div id="objectproperties">
          <h2>Object Properties</h2>
          <ul class="hlist">
           {% for obj_prop in object_properties %}
            <li>
               <a href="#{{ obj_prop.id }}" title="{{ obj_prop.uri }}">
                 <span>{{ obj_prop.label }}</span>
               </a>
            </li>
           {% endfor %}
          </ul>
         {% for obj_prop in object_properties %}
          <div id="{{ obj_prop.id }}" class="entity">
            <a name="{{ obj_prop.uri }}"></a>
            <h3>
            {{ obj_prop.label }} <sup title="object property" class="type-op">op</sup>
             <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#objectproperties">Object Property ToC</a></span>
            </h3>
            <p>
               <strong>IRI:</strong>
               {{ obj_prop.uri }}
            </p>
            <div class="comment">
               {% if obj_prop.definition %}
                {% for def in obj_prop.definition %}
                 <span class="markdown">{{ def }}</span>
                {% endfor %}
               {% endif %}
               {% if obj_prop.example %}
                {% for example in obj_prop.example %}
                 <span class="markdown">{{ example }}</span>
                {% endfor %}
               {% endif %}
            </div>
            {% if obj_prop.has_description %}
            <div class="description">
              <dl>
              {% if obj_prop.super_properties|length > 0 %}
               <dt>has super-properties</dt>
               {% for row in obj_prop.super_properties %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>
               {% endfor %}
              {% endif %}
              {% if obj_prop.domain|length > 0 %}
                <dt>has domain</dt>
                {% for row in obj_prop.domain %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if obj_prop.range|length > 0 %}
                <dt>has range</dt>
                {% for row in obj_prop.range %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if obj_prop.inverse_of|length > 0 %}
                <dt>is inverse of</dt>
                {% for row in obj_prop.inverse_of %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              </dl>
            </div>
            {% endif %}
          </div>
         {% endfor %}
         <div id="dataproperties">
           <h2>Data Properties</h2>
           <ul class="hlist">
           {% for data_type in data_types %}
            <li>
              <a href="#{{ data_type.id }}" title="{{ data_type.uri }}">{{ data_type.label }}</a>
            </li>
           {% endfor %}
           </ul>
           {% for data_type in data_types %}
           <div id="{{ data_type.id }}" class="entity">
            <a name="{{ data_type.uri }}"></a>
            <h3>
            {{ data_type.label }}
            <sup title="data property" class="type-dp">dp</sup>
            <span class="backlink"> back to <a href="#toc">ToC</a> or <a href="#dataproperties">Data Property ToC</a></span>
            </h3>
            <p>
              <strong>IRI:</strong>
               {{ data_type.uri }}
            </p>        
             {% if data_type.definition %}
               <div class="comment">
               {% for def in data_type.definition %}
                 <p><span class="markdown">{{ def }}</span></p>
               {% endfor %}
               </div>
             {% endif %}
             {% if data_type.comments %}
                <div class="comment">
                {% for comment in data_type.comments %}
                 <p><span class="markdown">{{ comment }}</span></p>
                {% endfor %}
                </div>
             {% endif %}
             {% if data_type.has_description %}
              <div class="description">
             {% if data_type.super_properties|length > 0 %}
               <dt>has super-properties</dt>
               {% for row in data_type.super_properties %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>
               {% endfor %}
              {% endif %}
              {% if data_type.domain|length > 0 %}
                <dt>has domain</dt>
                {% for row in data_type.domain %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if data_type.range|length > 0 %}
                <dt>has range</dt>
                {% for row in data_type.range %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              {% if data_type.inverse_of|length > 0 %}
                <dt>is inverse of</dt>
                {% for row in data_type.inverse_of %}
                <dd>
                  <span class="dotted" title="{{ row.uri }}">
                    {{ row.label|safe }}
                  </span>
                </dd>  
                {% endfor %}
              {% endif %}
              </div>
             {% endif %}
           </div>
           {% endfor %}
         </div>
        </div>
      </div>
      <div id="namespacedeclarations">
        <h2>Namespace Declarations <span class="backlink"> back to <a href="#toc">ToC</a></span></h2>
        <dl>
          <dt><em>default namespace</em></dt>
            <dd>http://performedmusicontology.org/ontology/</dd>
          <dt>bibframe</dt>
            <dd>http://id.loc.gov/ontologies/bibframe/</dd>
          <dt>dc</dt>
            <dd>http://purl.org/dc/elements/1.1/</dd>
          <dt>efrbroo</dt>
            <dd>http://erlangen-crm.org/efrbroo/</dd>
          <dt>owl</dt>
            <dd>http://www.w3.org/2002/07/owl#</dd>
          <dt>rdf</dt>
            <dd>http://www.w3.org/1999/02/22-rdf-syntax-ns#</dd>
          <dt>rdfs</dt>
            <dd>http://www.w3.org/2000/01/rdf-schema#</dd>
          <dt>regap</dt>
            <dd>http://metadataregistry.org/uri/profile/regap/</dd>
          <dt>skos</dt>
            <dd>http://www.w3.org/2004/02/skos/core#</dd>
          <dt>xsd</dt>
            <dd>http://www.w3.org/2001/XMLSchema#</dd>
        </dl>
    </div>
   </body>
</html>""")

In [16]:
def check_for_label(object_, graph):
    label = None
    if str(object_) in lookups:
        label = lookups[str(object_)]
    if not label:
        label = graph.value(subject=object_, predicate=rdflib.RDFS.label)
        if label:
            label = str(label)
    if label and str(object_).startswith("http://performedmusicontology.org"):
        label = create_link_super(object_, label)
    return label
        
def create_link_super(object_, label):
    ident = str(object_).split("/")[-1]
    return f"""<a href="#{ident}" title="{object_}">{label}</a>
    <sup title="object property" class="type-op">op</sup>"""

In [14]:
lookups = {
    "http://id.loc.gov/ontologies/bibframe/Contribution": "contribution",
    "http://id.loc.gov/ontologies/bibframe/derivativeOf": "derivative of",
    "http://id.loc.gov/ontologies/bibframe/Event": "event",
    "http://id.loc.gov/ontologies/bibframe/eventContent": "event content",
    "http://id.loc.gov/ontologies/bibframe/eventContentOf": "event content of",
    "http://id.loc.gov/ontologies/bibframe/hasDerivative": "has derivative",
    "http://id.loc.gov/ontologies/bibframe/Identifier": "identifier",
    "http://id.loc.gov/ontologies/bibframe/Organization": "organization",
    "http://id.loc.gov/ontologies/bibframe/referencedBy": "referenced by",
    "http://id.loc.gov/ontologies/bibframe/references": "references",
    "http://id.loc.gov/ontologies/bibframe/relatedTo": "related to",
    "http://id.loc.gov/ontologies/bibframe/Work": "work"
}

In [32]:
def build_info(subj: rdflib.URIRef):
    class_info = {
        "id": str(subj).split("/")[-1],
        "uri": str(subj),
    }
    label = pmo_graph.value(subject=subj, predicate=rdflib.RDFS.label)
    if label:
        class_info["label"] = str(label)
    else:
        class_info["label"] = class_info["id"]
    definition = pmo_graph.value(subject=subj, predicate=SKOS.definition)
    if definition:
        class_info["definition"] = str(definition)
    examples = pmo_graph.objects(subject=subj, predicate=SKOS.example)
    if examples:
        class_info["example"] = [str(i) for i in examples]
    return class_info

def add_superclasses(subject_uri: str) -> list:
    subj = rdflib.URIRef(subject_uri)
    super_classes = []
    sub_class_of = pmo_graph.objects(subject=subj, predicate=rdflib.RDFS.subClassOf)
    if sub_class_of:
        for super_class in sub_class_of:
            label = check_for_label(super_class, pmo_graph)
            if not label:
                raise ValueError(f"{subj} super-class {super_class} doesn't have a label")
            output = {
                "uri": str(super_class),
                "label": label
            }
            super_classes.append(output)
    return super_classes

def generate_description(subject, graph, all_obj_props):
    object_prop_description = { 
        "super_properties": [],
        "domain": [],
        "range": [],
        "inverse_of": [],
        "has_description": False,
    }
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.subPropertyOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s {obj} super properties not found in lookups")
        object_prop_description["has_description"] = True
        object_prop_description["super_properties"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.domain):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s domain of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["domain"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.range):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s range of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["range"].append(
            {
                "uri": str(obj),
                "label": label,
            }
        )
    for obj in graph.objects(subject=subject, predicate=OWL.inverseOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject} inverseOf {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["inverse_of"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    return object_prop_description

In [35]:
classes = {}
for subject in pmo_classes:
    subject_key = str(subject)
    classes[subject_key] = build_info(subject)
    super_classes = add_superclasses(subject_key)
    if len(super_classes) > 0:
        classes[subject_key]["superClass"] = add_superclasses(subject_key)

## Object Properties

In [41]:
def generate_description(subject, graph, all_obj_props):
    object_prop_description = { 
        "super_properties": [],
        "domain": [],
        "range": [],
        "inverse_of": [],
        "has_description": False,
    }
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.subPropertyOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s {obj} super properties not found in lookups")
        object_prop_description["has_description"] = True
        object_prop_description["super_properties"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.domain):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s domain of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["domain"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    for obj in graph.objects(subject=subject, predicate=rdflib.RDFS.range):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject}'s range of {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["range"].append(
            {
                "uri": str(obj),
                "label": label,
            }
        )
    for obj in graph.objects(subject=subject, predicate=OWL.inverseOf):
        label = check_for_label(obj, graph)
        if not label:
            raise ValueError(f"{subject} inverseOf {obj} not found")
        object_prop_description["has_description"] = True
        object_prop_description["inverse_of"].append(
            {
                "uri": str(obj),
                "label": label
            }
        )
    return object_prop_description

In [38]:
object_properties = [s for s in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.ObjectProperty)]

In [42]:
objects_props = {}
for subj in object_properties:
    object_property = {
      "id": str(subj).split("/")[-1],
      "uri": str(subj)
    }
    object_property.update(generate_description(subj, pmo_graph, objects_props))
    objects_props[str(subj)] = object_property
    label = pmo_graph.value(subject=subj, predicate=rdflib.RDFS.label)
    if not label:
        object_property["label"] = str(subj).split("/")[-1]
    else:
        object_property["label"] = str(label)
    definition = [str(s) for s in pmo_graph.objects(subject=subj, predicate=SKOS.definition)]
    if len(definition) > 0:
        object_property["definition"] = definition

## Data Types

In [45]:
data_types = {}
for subject in pmo_graph.subjects(predicate=rdflib.RDF.type, object=OWL.DatatypeProperty):
    subject_uri = str(subject)
    ident = subject_uri.split("/")[-1]
    data_type = {
        "id": ident,
        "uri": subject_uri
    }
    label = pmo_graph.value(subject=subject, predicate=rdflib.RDFS.label)
    if label:
        data_type["label"] = str(label)
    else:
        data_type["label"] = ident
    data_type.update(generate_description(subj, pmo_graph, data_types))
    definition = pmo_graph.objects(subject=subject, predicate=SKOS.definition)
    if definition:
        data_type["definition"] = [str(s) for s in definition]
    comments = pmo_graph.objects(subject=subject, predicate=rdflib.RDFS.comment)
    if comments:
        data_type["comments"] = [str(s) for s in comments]
    data_types[subject_uri] = data_type

## Generate PMO Home Page

In [46]:
rendered_pmo = template.render(classes = classes.values(), 
                               object_properties=objects_props.values(),
                               data_types=data_types.values())

In [51]:
with (version_2_0 / "Ontology/PMO.html").open("w") as fo:
    fo.write(rendered_pmo)